In [ ]:
import cv2
import matplotlib.pyplot as plt 
import math
from matplotlib.lines import Line2D
import numpy as np
from math import sqrt
import pandas as pd
from sklearn.cluster import OPTICS

In [ ]:
import imutils
def detect_fingers_final(image, title='default', bad_pict=False):
    point_between = []

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get binary image
    _, thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in the thresholded image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # Find the largest contour
    hand = max(cnts, key=cv2.contourArea)

    x,y,w,h = cv2.boundingRect(hand)

    # Calculate average color inside the bounding rectangle
    pixels= 1.0 * image[y:y + h, x:x + w].shape[0] * image[y:y + h,x:x + w].shape[1]
    channel_1 = np.sum(image[y:y + h, x:x + w][:,:,0]) / pixels
    channel_2 = np.sum(image[y:y + h,x:x + w][:,:,1]) / pixels
    channel_3 = np.sum(image[y:y + h,x:x + w][:,:,2]) / pixels

    av_color = np.mean(np.array([channel_1,channel_2,channel_3]).astype(np.uint8))

    # Calculate area and perimeter of the hand contour
    sq = cv2.contourArea(hand)
    perimeter = cv2.arcLength(hand,True)

    n = hand.copy()

    # Approximate the hand contour
    epsilon = 0.0005 * cv2.arcLength(n,True)
    approx= cv2.approxPolyDP(n,epsilon,True)

    # Simplify contour with Ramer-Douglas-Peucker algorithm
    epsilon = 0.01*cv2.arcLength(n,True)
    approx = cv2.approxPolyDP(n,epsilon,True)

    # Now use the simplified contour
    hull = cv2.convexHull(approx, returnPoints=True)
    cv2.drawContours(image, [hull], -1, (0, 255, 255), 2)

    hull_1 = cv2.convexHull(approx, returnPoints=False)
    defects = cv2.convexityDefects(approx, hull_1) #defects, which are the local maximum deviations of hull from contours

    l = 0 # number of defects
    start_arr = []
    end_arr = []
    fingers = []
    if defects is not None:
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])

            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            p_p = (a + b + c) / 2
            ar = math.sqrt(p_p * (p_p - a) * (p_p - b) * (p_p - c))

            d = (2 * ar) / a
            porog = 87

            if bad_pict:
                porog = 100

            angle = math.acos((b**2 + c**2 - a**2) / (2 * b * c)) * 57
            if l < 4:
                if angle <= porog and d > 30:
                    l += 1
                    if l == 1:
                        cv2.circle(image, far, 8, [255,0,0], -1)
                        cv2.circle(image, start, 8, [255,0,0], -1)
                        col = 'red'
                    elif l == 2:
                        cv2.circle(image, far, 8, [255,255,0], -1)
                        cv2.circle(image, start, 8, [255,255,0], -1)
                        col = 'yellow'

                    elif l == 3:
                        cv2.circle(image, far, 8, [0,255,0], -1)
                        cv2.circle(image, start, 8, [0,255,0], -1)
                        col = 'green'

                    elif l == 4:
                        cv2.circle(image, far, 8, [0,0,255], -1)
                        cv2.circle(image, start, 8, [0,0,255], -1)
                        col = 'blue'

                    point_between.append(far)
                    start_arr.append(start)
                    end_arr.append(end)
                    cv2.circle(image, start, 8, [255,0,0], -1)
                    fingers.append(start)

            cv2.line(image,start, end, [0,255,255], 2)
    else:
        print("No convexity defects found")

    min_dist = 1000000

    # identifying the pinky finger among the detected points by ensuring the end point (e) is sufficiently
    # far from all detected start points of the fingers (s)
    for e in end_arr:
        k = 0
        for s in start_arr:
            if abs(s[0] - e[0]) > 30 or abs(s[1] - e[1]) > 30:
                k += 1
        if k == 4:
            cv2.circle(image, e, 8, [255,127,0], -1)
            fingers.append(e)
            break

    return fingers, point_between, hull, end_arr, sq, av_color, perimeter
    # Returned values:
    # fingers: A list of points representing the detected finger tips. Each point is a tuple of (x, y) coordinates.
    # point_between: A list of points representing the spaces between the detected fingers. Each point is a tuple of (x, y) coordinates.
    # hull: The convex hull of the approximated hand contour. It is a numpy array of points that form the convex boundary around the hand.
    # end_arr: A list of end points of the convexity defects. Each point is a tuple of (x, y) coordinates.
    # sq: The area of the hand contour.
    # av_color: The average color inside the bounding rectangle of the hand.
    # perimeter: The perimeter of the hand contour.

In [ ]:
def draw_cont(fingers, point_between, image,title):
    dist_arr = {}
    dist_p = {}
    # Calculate distances from each finger to the 5th finger (if there are at least 5 fingers)
    for finger in fingers[:4]:
         if len(fingers) >= 5:
            dist_arr[finger] = dist(finger, fingers[4])

    # Calculate distances from each point between fingers to the 5th finger (if there are at least 5 fingers)
    for point in point_between:
        if len(fingers) >= 5:
            dist_p[point] = dist(point, fingers[4])

    # Create new lists of the first 4 fingers and points between fingers
    new_f = fingers[:4]
    new_d = point_between

    new_f.sort(key=lambda x: dist_arr[x] if x in dist_arr else 0)
    new_d.sort(key=lambda x: dist_p[x] if x in dist_p else 0)

    lst = []

    # Add the 5th finger to the list (if there are at least 5 fingers)
    if len(fingers) >= 5:
        lst.append(fingers[4])

    # Add points between fingers and the first 4 fingers to the list
    for i in range(4):
        if i < len(new_d):
            lst.append(new_d[i])
        if i < len(new_f):
            lst.append(new_f[i])

    lst_x = []
    lst_y = []
    for elem in lst:
        lst_x.append(elem[0])
        lst_y.append(elem[1])

    fig = plt.figure()
    ax = plt.subplot()

    line_1 = Line2D(lst_x, lst_y, color='magenta')
    ax.add_line(line_1)

    # Add the lines for widths
    line_lengths = []
    for i in range(4):
        if i < len(new_d) and i+1 < len(new_d):
            line_width = Line2D([new_d[i][0], new_d[i+1][0]], [new_d[i][1], new_d[i+1][1]], color='black')
            ax.add_line(line_width)
            length = dist(new_d[i], new_d[i+1])
            line_lengths.append(length)

    plt.title(title)
    
    plt.imshow(image[:,:,[2,1,0]])

    # Return the lists of points and the 5th finger (if there are at least 5 fingers)
    if len(fingers) >= 5:
        return new_f, new_d, fingers[4], line_lengths
    else:
        return new_f, new_d, None, None

In [ ]:
def center(image):
    # converting image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # getting binary image
    _, thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)
    # removing small white noises
    thresh = cv2.erode(thresh, None, iterations=60)

    # getting contours from the image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # selecting largest contour
    hand = max(cnts, key=cv2.contourArea)
    # calculating the minimum area bounding rectangle for the largest contour
    rot_rect = cv2.minAreaRect(hand)

    # returning the center of the minimum area rectangle (as the center of the palm)
    return rot_rect[0]

In [ ]:
import os

pict_lst = [f for f in os.listdir('training') if f.endswith('.jpg')]
all_cent = []
for pict in pict_lst:
    image = cv2.imread(os.path.join('training', pict))
    # calculating the center of the palm in each image
    all_cent.append(center(image))

In [ ]:
def dist(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def extract_finger_widths(image, hand_contour):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [hand_contour], -1, 255, thickness=cv2.FILLED)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    finger_widths = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        finger_widths.append(w)
    
    finger_widths = sorted(finger_widths, reverse=True)[:5]
    
    while len(finger_widths) < 5:
        finger_widths.append(0)
    return finger_widths

In [ ]:
database = {}
square = {}
color = {}
perim  = {}
fin_cent = {}
betw_cent = {}
betw_betw = {}
fin_widths = {}

pict_lst.sort()

for i, pict in enumerate(pict_lst[:len(pict_lst) - 1]):
    image = cv2.imread(os.path.join('training', pict))
    f,p,_,_,sq, av_color, perimeter = detect_fingers_final(image, title=pict)
    # if the first detection does not find exactly 5 fingers and 4 points between, the function is called again
    if not(len(f) == 5 and len(p) == 4):
        f,p,_,_,sq, av_color, perimeter = detect_fingers_final(image, title=pict, bad_pict=True)
    fing, betw, last, widths = draw_cont(f,p, image, title=pict)

    # computing distances between finger points and points between them and storing them in database
    database[pict] = []
    if len(betw) > 0:
        if last is not None:
            database[pict].append(dist(last, betw[0]))
    for j in range(3):
        if j < len(fing) and j < len(betw):
            database[pict].append(dist(fing[j], betw[j]))
            if j + 1 < len(betw):
                database[pict].append(dist(fing[j], betw[j + 1]))
    if len(fing) > 3 and len(betw) > 3:
        database[pict].append(dist(fing[3], betw[3]))
    
    fin_cent[pict] = []
    # computing distances from the palm center (all_cent[i]) to the fingers and points between them
    if last is not None:
        fin_cent[pict].append(dist(all_cent[i], last))
    for j in range(4):
        if j < len(fing):
            fin_cent[pict].append(dist(all_cent[i], fing[j]))
        
    betw_cent[pict] = []
    for j in range(4):
        if j < len(betw):
            betw_cent[pict].append(dist(betw[j], all_cent[i]))
        
    betw_betw[pict] = []
    for j in range(3):
        if j + 1 < len(betw):
            betw_betw[pict].append(dist(betw[j], betw[j + 1]))
    
    fin_widths[pict] = []
    for j in range(3):
        fin_widths[pict].append(widths[j])
        
    square[pict] = sq
    color[pict] = av_color
    perim[pict] = perimeter

In [ ]:
def euclidean_distance(X, Y):
    X_sum = np.sum(X * X, axis=1)
    Y_sum = np.sum(Y * Y, axis=1)
    XY_sum = np.dot(X, Y.T)

    X_new = np.tile(X_sum[:, np.newaxis], (1, Y.shape[0]))
    Y_new = np.tile(Y_sum[:, np.newaxis], (1, X.shape[0])).T

    return np.sqrt(np.maximum(X_new - 2 * XY_sum + Y_new, 0))

In [ ]:
#CREATE MATRIX WITH ALL FEATURES
num = {}
rev = {}
i = 0

matr = np.zeros((len(database), 28))
#first - main vector length - database
for key in database: #0 - 7
    num[key] = i
    rev[i] = key
    # Pad the array with zeros if its length is less than 8
    padded_array = np.pad(database[key], (0, 8 - len(database[key])), 'constant')

    # Now assign the padded_array to the slice of matr
    if i < matr.shape[0]:
        matr[i, :8] = padded_array
    i += 1
i = 0  
#second - new features: from center to fingers - fin_cent
for key in fin_cent: #8 - 12
    # Pad the array with zeros if its length is less than 5
    padded_array = np.pad(fin_cent[key], (0, 5 - len(fin_cent[key])), 'constant')

    # Now assign the padded_array to the slice of matr
    if i < matr.shape[0]:
        matr[i, 8:13] = padded_array
    i += 1
i = 0
#third - new features: from center to betws - betw_cent   
for key in betw_cent: #13 - 16
    # Pad the array with zeros if its length is less than 4
    padded_array = np.pad(betw_cent[key], (0, 4 - len(betw_cent[key])), 'constant')

    # Now assign the padded_array to the slice of matr
    if i < matr.shape[0]:
        matr[i, 13:17] = padded_array
    i += 1
i = 0
#forth- new features: from betws to betws - betw_betw
for key in betw_betw: #17 - 21
    # Pad the array with zeros if its length is less than 3
    padded_array = np.pad(betw_betw[key], (0, 3 - len(betw_betw[key])), 'constant')

    # Now assign the padded_array to the slice of matr
    if i < matr.shape[0]:
        matr[i, 17:20] = padded_array
    i += 1
i = 0   
#fifth - square
for key in square: #21
    if i < matr.shape[0]:
        matr[i, 20] = square[key]
    i += 1
i = 0


def calculate_distance_between_points(point1, point2):
    distance = np.linalg.norm(np.array(point1) - np.array(point2))
    return distance

def find_two_closest_fingers(betw_point, finger_points):
    distances = [(fp, calculate_distance_between_points(betw_point, fp)) for fp in finger_points]
    distances.sort(key=lambda x: x[1])
    return distances[:2]

betw_points = {}
for pict in pict_lst[:len(pict_lst) - 1]:
    image = cv2.imread(os.path.join('training', pict))
    fingers, point_between, _, _, _, _, _ = detect_fingers_final(image, title=pict)
    betw_points[pict] = point_between

finger_points = {}
for pict in pict_lst[:len(pict_lst) - 1]:
    image = cv2.imread(os.path.join('training', pict))
    fingers, _, _, _, _, _, _ = detect_fingers_final(image, title=pict)
    finger_points[pict] = fingers

#sixth - distances between finger point and two closest fingers
for key in betw_points: #22 - 23
    betw_point = betw_points[key]
    fingers = finger_points[key]
    closest_fingers = find_two_closest_fingers(betw_point, fingers)

    if i < matr.shape[0]:
        matr[i, 21] = closest_fingers[0][1]
        matr[i, 22] = closest_fingers[1][1]
    i += 1

i = 0
#seventh - finger widths
for key in fin_widths: #23 - 25
    # Pad the array with zeros if its length is less than 3
    padded_array = np.pad(fin_widths[key], (0, 3 - len(fin_widths[key])), 'constant')
    # Now assign the padded_array to the slice of matr
    if i < matr.shape[0]:
        matr[i, 23:26] = padded_array
    i += 1
i = 0  

In [ ]:
def count_euclidean(matr, idx=np.arange(0, 26), norm=False, num_neighbors=5):
    matr = matr[:,idx]
    if norm:
        matr = (matr - np.mean(matr, axis=0)) / np.std(matr, axis=0)

    dist_1 = euclidean_distance(matr, matr)
    for j in range(99):
        dist_1[j, j] = 1000000

    dist_3 = np.argsort(dist_1, axis=1)[:,:num_neighbors]

    return dist_1, dist_3

In [ ]:
def off_ticks(axes):
    axes.set_xticks([])
    axes.set_yticks([])
    
def find(pict, num, rev, dist_1, dist_3):
    neighbors = dist_3[num[pict]]
    distances = dist_1[num[pict], neighbors]

    # Ignore neighbors with distance 0
    valid_neighbors = neighbors[distances != 0]
    valid_distances = distances[distances != 0]

    original_person = pict.split('_')[0]
    valid_neighbors_same_person = [neighbor for neighbor in valid_neighbors if rev[neighbor].split('_')[0] == original_person]
    valid_distances_same_person = [distance for neighbor, distance in zip(valid_neighbors, valid_distances) if rev[neighbor].split('_')[0] == original_person]

    # If no valid neighbors of the same person are found, skip this image
    if not valid_neighbors_same_person:
        return

    gridsize = (1, len(valid_neighbors_same_person) + 1)
    fig = plt.figure(figsize=(15, 4))
    ax1 = plt.subplot2grid(gridsize, (0, 0))
    off_ticks(ax1)

    ax1.imshow(cv2.imread(os.path.join('training', pict))[:,:,[2,1,0]])
    ax1.set_title(f'Original: {pict}')

    for i, (neighbor, distance) in enumerate(zip(valid_neighbors_same_person, valid_distances_same_person)):
        if i + 1 < gridsize[1]:  # Add this check
            ax = plt.subplot2grid(gridsize, (0, i + 1))
            off_ticks(ax)
            p = rev[neighbor]
            d = distance
            ax.imshow(cv2.imread(os.path.join('training', p))[:,:,[2,1,0]])
            ax.set_title(f'Neigh_{i + 1}: {p}, dist: {d:.3f}')

In [ ]:
idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25]
dist_1, dist_3 = count_euclidean(matr, idx, True, num_neighbors=5)
for pict in pict_lst[:len(pict_lst) - 1]:
    find(pict, num, rev, dist_1, dist_3)

In [ ]:
def calculate_accuracy(dist_3, rev):
    accurate_predictions = set()

    # Create a set to store the unique persons
    unique_persons = set()

    for i in range(dist_3.shape[0]):
        neighbors = dist_3[i]
        original_person = rev[i].split('_')[0]
        unique_persons.add(original_person)

        for neighbor in neighbors:
            neighbor_person = rev[neighbor].split('_')[0]
            if original_person == neighbor_person:
                accurate_predictions.add(original_person)
                break

    total_predictions = len(unique_persons)

    accuracy = len(accurate_predictions) / total_predictions
    return accuracy

# Zmieniamy num_neighbors na 8 w funkcji count_euclidean
num_images = len(pict_lst[:len(pict_lst) - 1])  # liczba zdjęć
num_neighbors = int(num_images * 0.1)  # 10% liczby zdjęć
dist_1, dist_3 = count_euclidean(matr, idx, True, num_neighbors=5)
accuracy = calculate_accuracy(dist_3, rev)
print(f"Accuracy: {accuracy}")